# Feature based time series classification in aeon

Feature based classifiers are a popular recent theme in TSC. The feature
based classifiers we provide are simply pipelines of transform and classifier. They
extract descriptive statistics as features from time series to be used in
classifiers. Several tool-kits exist for extracting features.


<img src="./img/pipeline.png" width="800" alt="Pipeline classifier.">


## Imports and Load Data

In [5]:
from sklearn import metrics

from aeon.classification.feature_based import Catch22Classifier, FreshPRINCE
from aeon.datasets import load_basic_motions, load_italy_power_demand
from aeon.registry import all_estimators
from aeon.transformations.panel.catch22 import Catch22

X_train, y_train = load_italy_power_demand(split="train")
X_test, y_test = load_italy_power_demand(split="test")
X_test = X_test[:50]
y_test = y_test[:50]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train_mv, y_train_mv = load_basic_motions(split="train")
X_test_mv, y_test_mv = load_basic_motions(split="test")

X_train_mv = X_train_mv[:20]
y_train_mv = y_train_mv[:20]
X_test_mv = X_test_mv[:20]
y_test_mv = y_test_mv[:20]

print(X_train_mv.shape, y_train_mv.shape, X_test_mv.shape, y_test_mv.shape)
# search for all classifiers that can handle multivariate time series. This will
# give some UserWarnings if soft dependencies are not installed. Rerun to remove
# warnings.
all_estimators("classifier", filter_tags={"algorithm_type": "feature"})

[('Catch22Classifier',
  aeon.classification.feature_based._catch22_classifier.Catch22Classifier),
 ('FreshPRINCE', aeon.classification.feature_based._fresh_prince.FreshPRINCE),
 ('SignatureClassifier',
  aeon.classification.feature_based._signature_classifier.SignatureClassifier),
 ('SummaryClassifier',
  aeon.classification.feature_based._summary_classifier.SummaryClassifier),
 ('TSFreshClassifier',
  aeon.classification.feature_based._tsfresh_classifier.TSFreshClassifier)]

### Catch22 and the Catch22Classifier

The highly comparative time-series analysis (hctsa) [1] toolbox can create over 7700
features for exploratory time series analysis. The canonical time series
characteristics (catch22) [2] are 22 hctsa features determined to be the most
discriminatory of the full  set. The catch22 were chosen by an evaluation on the UCR
datasets. The hctsa features were initially pruned, removing those which are
sensitive to mean and variance and any which could not be calculated on over 80\% of
the UCR  datasets. A feature evaluation was then performed based on predictive
performance. Any features which performed below a threshold were removed. For the
remaining features, a hierarchical clustering was performed on the correlation matrix
 to remove redundancy. From each of the 22 clusters formed, a single feature was
 selected, taking into account balanced accuracy, computational efficiency and
 interpretability. The catch22 features cover a wide range of concepts such as basic
 statistics of data series values, linear correlations, and entropy. Reported results
  for catch22 build a decision tree classifier after applying the transform to each
  data series.

The Catch22Classifier is simply a convenient wrapper for a pipelin of a Catch22
transformation and a sklearn random forest classifier by default.

In [6]:
c22 = Catch22()
x_trans = c22.fit_transform(X_train)
x_trans.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1.158630,-0.217227,8.0,0.291667,-0.625000,3.0,6.0,0.468052,0.589049,0.836755,...,3.0,1.000000,5.0,1.778748,0.750000,0.240598,NaN,NaN,0.040000,0.0
1,0.918162,-0.214762,15.0,0.208333,-0.666667,4.0,8.0,0.702775,0.196350,0.666160,...,4.0,0.869565,5.0,1.730238,0.500000,0.388217,NaN,NaN,0.111111,0.0
2,-0.273180,-0.085856,4.0,0.875000,0.250000,2.0,5.0,0.310567,0.589049,0.865073,...,2.0,0.913043,5.0,1.836012,0.666667,0.089104,NaN,NaN,0.034014,0.0
3,0.048411,-0.450080,13.0,0.166667,-0.625000,4.0,10.0,0.804047,0.196350,0.648309,...,4.0,0.869565,6.0,1.605420,0.666667,0.332436,NaN,NaN,0.111111,0.0
4,0.426379,0.572566,16.0,0.291667,-0.666667,4.0,7.0,0.675485,0.196350,0.657946,...,4.0,0.913043,6.0,1.730238,0.500000,0.318405,NaN,NaN,0.111111,0.0


In [7]:
c22cls = Catch22Classifier()
c22cls.fit(X_train, y_train)
c22_preds = c22cls.predict(X_test)
metrics.accuracy_score(y_test, c22_preds)

0.9

### TSFresh and the FreshPRINCE

Time Series Feature Extraction based on Scalable Hypothesis Tests (TSFresh) is a
collection of just under 800 features extracted from time series. While the features
can be used on their own, a feature selection method called FRESH is provided to
remove irrelevant features. FRESH considered each feature using multiple hypotheses
tests, including Fisher's exact test, the Kolmogorov-Smirnov test and the Kendal rank test.
    The Benjamini-Yekutieli procedure is then used to control the false discovery
    rate caused by comparing multiple hypotheses and features simultaneously.

In [8]:
fp = FreshPRINCE()
fp.fit(X_train, y_train)
fp_preds = c22cls.predict(X_test)
metrics.accuracy_score(y_test, fp_preds)

0.9

### Signatures and the SignatureClassifier

Generalised signatures are a set of feature extraction techniques primarily for
multivariate time series based on rough path theory. We specifically look at the
generalised signature method~\citep{morrill20generalised} and the accompanying
canonical signature pipeline. Signatures are collections of ordered cross-moments.
The pipeline begins by applying two augmentations by default. The basepoint
augmentation simply adds a zero at the beginning of the time series, making the
signature sensitive to translations of the time series. The time augmentation adds
the series timestamps as an extra coordinate to guarantee each signature is unique
and obtain information about the parameterisation of the time series. A hierarchical
dyadic window is run over the series, with the signature transform being applied to
each window. The output for each window is then concatenated into a feature vector.
Signatures requires the installation of the soft dependency esig.

### Comparing performance: coming soon

## References

[1]
[2] Christ, Maximilian, et al. "Time series feature extraction on basis of
        scalable hypothesis tests (tsfresh–a python package)." Neurocomputing 307
        (2018): 72-77.
        https://www.sciencedirect.com/science/article/pii/S0925231218304843


